In [2]:
from google.colab import drive
import os

drive.mount('/content/drive/')  

Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
!pip install --upgrade implicit

     |████████████████████████████████| 18.5 MB 1.2 MB/s 


In [5]:
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

In [6]:
path = '/content/drive/MyDrive/Colab Notebooks/data/'
csv_train = f'{path}transactions_train.csv'

csv_users = f'{path}customers.csv'
csv_items = f'{path}articles.csv'

df = pd.read_csv(csv_train, dtype={'article_id': str}, parse_dates=['t_dat'])


In [18]:
dfu = pd.read_csv(csv_users)
dfi = pd.read_csv(csv_items, dtype={'article_id': str})

In [8]:
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [19]:
dfu.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [10]:
dfi.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [13]:
start = '2020-01-21' 
end =  str(df['t_dat'].max()).split(' ')[0]

In [14]:
df = df[(df['t_dat'] >= start) & (df['t_dat'] <= end)]
df.shape

(10302891, 5)

In [20]:
Users= dfu['customer_id'].unique().tolist()

Articles = dfi['article_id'].unique().tolist()

user_ids = dict(list(enumerate(Users)))
item_ids = dict(list(enumerate(Articles)))

In [21]:
map={u: id for id, u in user_ids.items()}
df['user_id']=df['customer_id'].map(map)
map={i: id for id, i in item_ids.items()}
df['item_id']=df['article_id'].map(map)

In [25]:
r=df['user_id'].values
c=df['item_id'].values
data=np.ones(df.shape[0])
sparseMatrix=coo_matrix((data, (r, c)), shape=(len(Users), len(Articles)))

In [26]:
sparseMatrix.shape

(1371980, 105542)

In [31]:
def getCooMatrix(df,Users,Articles):
    #get the customers and articles sparse matrix
    r=df['user_id'].values
    c=df['item_id'].values
    data=np.ones(df.shape[0])
    coo=coo_matrix((data, (r, c)), shape=(len(Users), len(Articles)))
    return coo

In [34]:
def get_Train_GT(df, validation_days=7):
    #get the ground truth
    validation_cut = df['t_dat'].max() - pd.Timedelta(validation_days)

    df_train = df[df['t_dat'] < validation_cut]
    df_GT = df[df['t_dat'] >= validation_cut]
   


    coo_train=getCooMatrix(df_train,Users,Articles)
    coo_GT=getCooMatrix(df_GT,Users,Articles)

    train=coo_train.tocsr()
    GT=coo_GT.tocsr()
    
    return {'coo_train': coo_train,'train': train,'GT': GT}

In [35]:
def validate(matrices, factors=200, iterations=20):
    #run a expriment, factors: model para, 
    # get train and test data
    coo_train, train, GT = matrices['coo_train'], matrices['train'], matrices['GT']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors,iterations=iterations,regularization=0.01)
    model.fit(coo_train, show_progress=True)
    
    

    # the evaluation function from implicit
    map12 = mean_average_precision_at_k(model, train, GT, K=12, show_progress=True, num_threads=2)
    print('Factors:',factors,' Iters:',iterations,' mean_average_precision', map12)


    return map12

In [33]:
matrices = get_Train_GT(df)

Main progress:

In [ ]:
best_score = 0
best_iter=-1
best_fac=-1
for iterations in [10,100,200]:
  for factors in [50,100,200,300]:
    map12 = validate(matrices, factors, iterations)
    if map12 > best_score:
      best_score = map12
      best_iter=iterations
      best_fac=factors

print('The best result: ',best_score,' best factors:',best_fac,' best_iter: ',best_iter)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10528 [00:00<?, ?it/s]

Factors: 50  Iters: 10  mean_average_precision 0.00350766618828997


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10528 [00:00<?, ?it/s]

...

  0%|          | 0/10528 [00:00<?, ?it/s]

Factors: 300  Iters: 10  mean_average_precision 0.004136326697829363


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10528 [00:00<?, ?it/s]

Factors: 50  Iters: 100  mean_average_precision 0.0035056010169737537
